In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving merged_df3.5.csv to merged_df3.5.csv


In [ ]:
# Read the CSV file
import io
nba_df = pd.read_csv(io.BytesIO(uploaded['merged_df3.5.csv']))

nba_df.head()

,game_date_est,game_id,home_team,away_team,home_team_wins,pts_home,pts_away,pts_comb_actual,book_name_over,total1_over,over_odds,book_name_under,total1_under,under_odds,over_successful,under_successful
0,9/30/2017,11700001,GSW,DEN,0,102,108,210,Intertops,223.0,-110,Pinnacle Sports,222.5,108,0,1
1,9/30/2017,11700002,LAL,MIN,0,99,108,207,Pinnacle Sports,214.5,-110,Bookmaker,216.0,-102,0,1
2,11/1/2017,21700111,NYK,HOU,0,97,119,216,5Dimes,214.0,-107,Pinnacle Sports,215.5,-102,1,0
3,11/1/2017,21700107,PHI,ATL,1,119,109,228,Pinnacle Sports,209.0,-104,5Dimes,209.0,-105,1,0
4,11/1/2017,21700108,WAS,PHX,0,116,122,238,Pinnacle Sports,227.5,-105,Heritage,228.0,-105,1,0


In [ ]:
# Drop non-beneficial columns if applicable
nba_df.drop(columns=['under_successful'], inplace=True)
nba_df.drop(columns=['under_odds'], inplace=True)
nba_df.drop(columns=['total1_under'], inplace=True)
nba_df.drop(columns=['pts_away'], inplace=True)
nba_df.drop(columns=['pts_home'], inplace=True)
nba_df.drop(columns=['home_team_wins'], inplace=True)

nba_df.head()

,game_date_est,game_id,home_team,away_team,pts_comb_actual,book_name_over,total1_over,over_odds,book_name_under,over_successful
0,9/30/2017,11700001,GSW,DEN,210,Intertops,223.0,-110,Pinnacle Sports,0
1,9/30/2017,11700002,LAL,MIN,207,Pinnacle Sports,214.5,-110,Bookmaker,0
2,11/1/2017,21700111,NYK,HOU,216,5Dimes,214.0,-107,Pinnacle Sports,1
3,11/1/2017,21700107,PHI,ATL,228,Pinnacle Sports,209.0,-104,5Dimes,1
4,11/1/2017,21700108,WAS,PHX,238,Pinnacle Sports,227.5,-105,Heritage,1


In [ ]:
# Determine the number of unique values in each column.
nba_df.nunique()

game_date_est       198
game_id            1210
home_team            30
away_team            30
pts_comb_actual     106
book_name_over        9
total1_over          92
over_odds            21
book_name_under       8
over_successful       2
dtype: int64

In [ ]:
# Look at actual game points value counts for binning
nba_df['pts_comb_actual'].value_counts()

215    34
207    33
205    27
213    27
211    27
       ..
255     1
250     1
158     1
171     1
166     1
Name: pts_comb_actual, Length: 106, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of actual game points to be replaced
# use the variable name `pts_types_to_replace`
pts_types = pd.DataFrame(nba_df['pts_comb_actual'].value_counts()).reset_index()

pts_types_to_replace = []

for index, row in pts_types.iterrows():
    if row['pts_comb_actual'] < 10:
        pts_types_to_replace.append(row['index'])
pts_types_to_replace


# Replace in dataframe
for pts_types in pts_types_to_replace:
    nba_df['pts_comb_actual'] = nba_df['pts_comb_actual'].replace(pts_types,"Other")

# Check to make sure binning was successful
nba_df['pts_comb_actual'].value_counts()

Other    189
215       34
207       33
213       27
206       27
205       27
211       27
217       26
192       25
208       25
209       25
223       25
204       24
202       24
218       24
221       24
224       23
201       23
226       23
227       22
225       21
193       21
210       21
233       20
187       20
220       19
212       19
203       19
200       19
214       18
195       17
197       17
231       17
191       16
190       16
196       16
232       15
219       15
234       15
198       14
189       14
228       14
216       14
237       14
241       13
194       13
240       13
229       13
199       13
188       13
230       11
236       11
235       11
242       11
Name: pts_comb_actual, dtype: int64

In [ ]:
# Look at point-average-last10 value counts for binning
# Try other columns
oo_counts = nba_df['over_odds'].value_counts()
oo_counts

-105    555
-103    138
-102    116
-110     66
-104     55
-101     48
-107     43
-108     37
-100     37
 100     26
 102     22
-115     13
-106     13
 101     12
-109      8
 103      8
 104      6
 105      4
-111      1
 106      1
 109      1
Name: over_odds, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of YYYY to be replaced total1_over
# use the variable name `yyyy_to_replace`
oo_types = pd.DataFrame(nba_df['total1_over'].value_counts()).reset_index()

oo_to_replace = []

for index, row in oo_types.iterrows():
    if row['total1_over'] < 5:
        oo_to_replace.append(row['index'])
oo_to_replace

# Replace in dataframe
for oo_types in oo_to_replace:
    nba_df['total1_over'] = nba_df['total1_over'].replace(oo_types,"Other")

# Check to make sure binning was successful
nba_df['total1_over'].value_counts()

Other    46
208.0    42
217.0    37
212.0    37
210.0    34
         ..
198.5     6
227.5     6
198.0     6
231.0     6
228.5     5
Name: total1_over, Length: 69, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
nba_df = pd.get_dummies(nba_df)

In [ ]:
# Split our preprocessed data into our features and target arrays
y = nba_df['over_successful']
X = nba_df.drop(columns=['over_successful'])

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow import keras
from tensorflow.keras import layers
num_input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(layers.Dense(units = 100, activation = 'relu', input_dim = num_input_features))

# Second hidden layer
nn.add(layers.Dense(units = 50, activation = 'elu'))

# Output layer
nn.add(layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               40100     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 45,201
Trainable params: 45,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# Train the model
fit_modelnn = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
29/29 [==============================] - 1s 2ms/step - loss: 0.7439 - accuracy: 0.5546
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 0.4565 - accuracy: 0.8269
Epoch 3/100
29/29 [==============================] - 0s 1ms/step - loss: 0.2856 - accuracy: 0.9283
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1619 - accuracy: 0.9735
Epoch 5/100
29/29 [==============================] - 0s 1ms/step - loss: 0.0860 - accuracy: 0.9923
Epoch 6/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0426 - accuracy: 0.9989
Epoch 7/100
29/29 [==============================] - 0s 1ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 8/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 9/100
29/29 [==============================] - 0s 1ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 10/100
29/29 [==============================] - 0s 1ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 11/

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

10/10 - 0s - loss: 1.8351 - accuracy: 0.6898 - 118ms/epoch - 12ms/step
Loss: 1.835059642791748, Accuracy: 0.6897689700126648
